In [1]:
import tweepy
import csv
import pandas as pd
import time
import sys
import itertools
from datetime import datetime
import sys
import networkx as nx
import plotly as py
import plotly.graph_objects as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Twitter API credentials
#consumer_key = "BmvoUUcOUXhPxRR8uRC2TgKoW"
#consumer_secret = "bNV6inRgeUSSVerytnnnTPveW8iM9GM0dwryZyiUKmYy436D1I"
#access_key = "2969993776-b9Ui7fVJjW7gYId2C0kSGo5mN4ki93HSGEn6jx0"
#access_secret = "N5ER33zjeIqfl5918MWTHLWbZzuBGfGL0FeSfNGvSsrvZ"

In [3]:
#OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret, 'access_token_key':access_key, 'access_token_secret':access_secret}
#auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Functions

In [5]:
def load_folder_as_dict(dates, prefix='data_10000/'):
    dict_df = dict()
    for i in range(len(dates) - 1):
        csv_name = prefix + 'data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
        df_name = 'df_' + str(dates[i])
        dict_df[df_name] = pd.read_csv(csv_name)
    return dict_df

In [7]:
def define_graphs(dict_df, stats=False):
    graph_dict = dict()
    for key in dict_df.keys():
        df = dict_df[key]
        G = nx.from_pandas_edgelist(
            df,
            source = 'username', 
            target = 'user_retweeted',
            create_using = nx.DiGraph())
        graph_dict[key] = G
        if stats:
            print('Nodes in RT network:', len(G.nodes()))
            print('Edges in RT network:', len(G.edges()), '\n')
    return graph_dict

In [8]:
def plot_graph(G):
    
    # Some node-level features
    centralScore = nx.betweenness_centrality(G)
    inScore = G.in_degree()
    outScore = G.out_degree()
    # Get a list of all nodeID in ascending order
    nodeID = G.nodes.keys()
    nodeID = sorted(nodeID)

    # Node label information available on hover.
    labels = []
    for nd in nodeID:
        labels.append(nd + "<br>" + "In-retweets: " + str(inScore[nd]) + "<br>" +
                      "Out-retweets: " + str(outScore[nd]) + "<br>" + "Centrality: " 
                      + str("%0.3f" % centralScore[nd]))

    # Type of graph layout:
    pos=nx.spring_layout(G)
    E=[tuple(e) for e in G.edges()]
    N=len(labels)
    axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title=''
              )
    width=800
    height=800
    layout=Layout(title= "Twitter Retweet Network with #confinement in Paris"+\
                  "<br> Date : XX/XX/2020",
        font= dict(size=12),
        showlegend=False,
        autosize=False,
        width=width,
        height=height,
        xaxis={'showgrid': False, 'zeroline': False, 'visible': False},
        yaxis={'showgrid': False, 'zeroline': False, 'visible': False},
        grid=None,
        hovermode='closest',
        annotations=[
               dict(
               showarrow=False,
                text='This igraph.Graph has the Kamada-Kawai layout',
                xref='paper',
                yref='paper',
                x=0,
                y=-0.1,
                xanchor='left',
                yanchor='bottom',
                font=dict(
                size=14
                )
                )
            ]
        )
    Xv=[pos[k][0] for k in G.nodes()]
    Yv=[pos[k][1] for k in G.nodes()]
    Xed=[]
    Yed=[]
    for edge in E:
        Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
        Yed+=[pos[edge[0]][1],pos[edge[1]][1], None]

    trace1=Scatter(x=Xed,
                   y=Yed,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   )
    trace2=Scatter(x=Xv,
                   y=Yv,
                   mode='markers',
                   name='net',
                   marker=dict(symbol='circle-dot',
                                 size=5,
                                 color='#01B7EF',
                                 line=dict(color='rgb(12,12,90)', width=0.5)
                                 ),
                   text=labels,
                   hoverinfo='text'
                   )

    #annot="Circular Layout"

    data1=[trace1, trace2]
    fig1=Figure(data=data1, layout=layout)
    fig1['layout']['annotations'][0]['text']=annot
    fig1.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })

    return fig1

In [10]:
dates = ['2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31',
        '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06']

In [11]:
dict_df = load_folder_as_dict(dates, prefix='data/')

In [12]:
graph_dict = define_graphs(dict_df)

In [83]:
key = list(graph_dict.keys())[0]
G = graph_dict[key]

In [ ]:
fig = plot_graph(G)
fig.show()